# Module

In [1]:
import argparse
from tqdm import tqdm_notebook as tq
import tqdm
import os, time, math, copy
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple
import matplotlib.pyplot as plt
import random
import time
import datetime

torch.set_printoptions(precision=8, linewidth=50000)
import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.6/dist-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Print Colors

In [2]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

In [6]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [7]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        #if fIn < self.fMin or fIn > self.fMax:
            #print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [8]:
def flp2fixTensor(fIn, iBW, iBWF):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())

# User Define Variables

In [9]:
data_path = '~/dataset'

In [10]:
parser = argparse.ArgumentParser(description='PyTorch for MNIST dataset')
parser.add_argument('--device', type=str, default='cpu', help='Device')
parser.add_argument('--shuffle', action='store_true', default=False, help='enables data shuffle')
parser.add_argument('--dataset', type=str, default='mnist', help='training dataset')
parser.add_argument('--data_path', type=str, default=data_path, help='path to MNIST')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=10, help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--optimizer', type=str, default='adam', help='optimizer')
parser.add_argument('--loss_func', type=str, default='cel', help='optimizer')
parser.add_argument('--quant_opt', type=str, default='asym', help='Type of Quantization')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=7, help='Number of Quantization Bits')
#parser.add_argument('--pretrained', type=bool, default=True, help='Pretrained Model')
parser.add_argument('--act_quant', type=bool, default=False, help='Activation Quantization')
parser.add_argument('--disp', type=bool, default=False, help='Display Model Information')
parser.add_argument('--bBW',type=int,default=7,help='bit number')
args = parser.parse_args(args=[])

# Preparing Data

In [11]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.device == 'cuda' else {}
if args.dataset == 'mnist':
	train_loader = torch.utils.data.DataLoader(
		dataset=datasets.MNIST(
			root=args.data_path,
			train=True,
			download=True,
			transform=transforms.ToTensor()
		),
		batch_size=args.batch_size,
		shuffle=args.shuffle,
		**kwargs
	)

	test_loader = torch.utils.data.DataLoader(
		dataset=datasets.MNIST(
			root=args.data_path,
			train=False,
			download=True,
			transform=transforms.ToTensor()
		),
		batch_size=args.batch_size,
		shuffle=args.shuffle,
		**kwargs
	)

# Build Model

In [12]:
class MLP(nn.Module):
	def __init__(self):
		super(MLP, self).__init__()
		self.flatten = nn.Flatten()
		self.fc1 = nn.Linear(28*28, 32)
		self.relu1 = nn.ReLU()
		self.fc2 = nn.Linear(32, 64)
		self.relu2 = nn.ReLU()
		self.fc3 = nn.Linear(64, 10)
		
	def forward(self, x):
		x = self.flatten(x)
		x = self.fc1(x)
		x = self.relu1(x)
		x = self.fc2(x)
		x = self.relu2(x)
		logits = self.fc3(x)
		return logits

In [13]:
def genOptimizer(model, args):
	if args.optimizer == 'sgd':
		optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
	if args.optimizer == 'adam':
		optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
	return optimizer

def genLossFunc(args):
	if args.loss_func == 'cel':
		loss_func = nn.CrossEntropyLoss()
	return loss_func

In [14]:
def train(train_loader, model, epoch, args):
	model.train()
	loss_func = genLossFunc(args)
	optimizer = genOptimizer(model, args)
	max_batch_index = int(np.floor(len(train_loader.dataset)/args.batch_size))
	running_loss = 0
	for batch_index, (image, label) in enumerate(tq(train_loader, desc='Train', leave=False)):
		image, label = image.to(args.device), label.to(args.device)
		pred = model(image)
		loss = loss_func(pred, label)
		running_loss += loss.item()#*image.size(0)

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
				
	print(f'Epoch {epoch+1:<3d}: Avg. Loss: {running_loss/len(train_loader.dataset):.4f}', end = '\t')

In [15]:
def test(test_loader, model, args):
	model.eval()
	with torch.no_grad():
		loss_func = genLossFunc(args)
		loss, correct = 0, 0
# 		for batch_index, (image, label) in enumerate(tq(test_loader, desc='Test', leave=False)):
		for batch_index, (image, label) in enumerate(test_loader):
			image, label = image.to(args.device), label.to(args.device)
			pred = model(image)
			loss += loss_func(pred, label).item()#*image.size(0)
			correct += (pred.argmax(1) == label).type(torch.int).sum().item()
	loss /= len(test_loader.dataset)
	correct_rate = 100 * correct / len(test_loader.dataset)
	print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%)')

In [16]:
def main(model):
	for epoch in range(args.epochs):
		train(train_loader, model, epoch, args)
		test(test_loader, model, args)
	print("Done!")
	return model

In [17]:
#if args.pretrained:
#    if os.path.isfile('preTrainedModel.pth'):
#        model = MLP().to(args.device)
#        model.load_state_dict(torch.load('preTrainedModel.pth'))
#        test(test_loader, model, args)
#    else:
#        model = main(MLP().to(args.device))
#        torch.save(model.state_dict(), 'preTrainedModel.pth')
#else:
model = main(MLP().to(args.device))
torch.save(model.state_dict(), 'preTrainedModel.pth')

Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 1  : Avg. Loss: 0.0072	Accuracy: 9253/10000 (92.5%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 2  : Avg. Loss: 0.0034	Accuracy: 9403/10000 (94.0%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

...

Epoch 8  : Avg. Loss: 0.0013	Accuracy: 9637/10000 (96.4%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 9  : Avg. Loss: 0.0012	Accuracy: 9641/10000 (96.4%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 10 : Avg. Loss: 0.0011	Accuracy: 9654/10000 (96.5%)
Done!


In [18]:
print(model.fc1.weight)

Parameter containing:
tensor([[-0.02683339, -0.01054859,  0.03032732,  ...,  0.01771713,  0.01221379, -0.02618989],
        [-0.01719414,  0.01315976,  0.01165506,  ...,  0.01930584,  0.02855700,  0.00489422],
        [ 0.01752516, -0.03049587, -0.01644560,  ..., -0.00689845, -0.01012152,  0.01757649],
        ...,
        [ 0.01941297,  0.02264481, -0.00148786,  ..., -0.00716893, -0.02862862,  0.00629433],
        [ 0.02069288, -0.02025052,  0.01866948,  ...,  0.01830390, -0.01152918,  0.00178513],
        [-0.03558008,  0.00286253,  0.03205480,  ...,  0.00888480, -0.02736063, -0.02299960]], requires_grad=True)


# SNG

In [19]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

In [20]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

In [21]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[5]),int(stream[6]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        self.b4 = stream[3]
        self.b5 = stream[4]
        self.b6 = stream[5]
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

In [22]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(7)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(7)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

In [23]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == '0'*args.full_bits :
        return "0"*((2**(args.bBW))+1)
    else :
        return sA

In [24]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if (args.bBW == args.frac_bits) :    
            lNUM = perm(lfsr[k])
        elif (args.bBW < args.frac_bits) :
            lNUM = perm(lfsr[k][:args.bBW])+(args.frac_bits-args.bBW)*"0"
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == '0'*args.full_bits :
        return "0"*((2**(args.bBW))+1)
    else :
        return sA

In [25]:
def SNGnumpy(fIn,lfsr):
    start = time.time()
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
    end = time.time()
    sec = (end-start)
    result_list = str(datetime.timedelta(seconds=sec)).split(".")
    #print(f'SNGnumpy : {result_list[0]}')                 
    return np.array(sList).reshape(fIn.size())

In [26]:
def SNGpnumpy(fIn,lfsr):
    start = time.time()
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
    end = time.time()
    sec = (end-start)
    result_list = str(datetime.timedelta(seconds=sec)).split(".")
    #print(f'SNGpnumpy : {result_list[0]}')                   
    return  np.array(sList).reshape(fIn.size())

In [27]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if a[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

In [28]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

In [29]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [30]:
def defSign1(nIn):
    if nIn[0] == '1' :
        return -1
    else :
        return 1

In [31]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    if nIn[0] == '1' :
        return n-1
    else :
        return n

In [32]:
def S2None(sIn,SF):
    s = defSign1(sIn)
    o = (CountOne1(sIn)/(2**args.bBW))*SF*s
    return o

In [33]:
def mulNumpy(aIn,bIn,aSF,wSF):
    start = time.time()
    mList = []
    for i in range(aIn.shape[0]):
        for j in range(bIn.T.shape[1]):
            sum = 0
            for k in range(aIn.shape[1]):
                sum += S2None(mul((aIn[i][k].astype(str)),(bIn.T)[k][j].astype(str)),aSF*wSF)
            mList.append(sum)
    end = time.time()
    sec = (end-start)
    result_list = str(datetime.timedelta(seconds=sec)).split(".")
    #print(f'mulNumpy : {result_list[0]}')
    return torch.tensor(mList).view(aIn.shape[0],bIn.T.shape[1])

## Find max, min 

In [34]:
def findMaxMin(data):
    start = time.time()
    
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    
    end = time.time()
    sec = (end-start)
    result_list = str(datetime.timedelta(seconds=sec)).split(".")
    #print(f'findMaxMin : {result_list[0]}')
    
    return SF

## Fixed model

In [35]:
def model2fix(model, args):
	for name, _ in model.named_parameters():
		exec(f'model.{name}.data = flp2fixTensor(model.{name}.data, {args.full_bits}, {args.frac_bits})')
	return model

In [36]:
def N2S2N(model, iX, iW, iB, args):
    lfsr = LFSRlist7()
    
    xSF = findMaxMin(iX)
    wSF = findMaxMin(iW)
    
    x = SNGnumpy(iX/xSF,lfsr)
    w = SNGpnumpy(iW/wSF,lfsr)
    
    sout = mulNumpy(x,w,xSF,wSF) + iB
    
    return sout

In [37]:
def quantFixForward(model, x, args):
    cmodel = copy.deepcopy(model).to(args.device)
    
    with torch.no_grad():
        i0 = cmodel.flatten(x)
        act0 = N2S2N(cmodel,i0,cmodel.fc1.weight,cmodel.fc1.bias,args)
        #print("act0 successed")
        
        i1 = model.relu1(act0)
        act1 = N2S2N(cmodel,i1,cmodel.fc2.weight,cmodel.fc2.bias,args)
        #print("act1 successed")
        
        i2 = cmodel.relu2(act1)
        act2 = N2S2N(cmodel,i2,cmodel.fc3.weight,cmodel.fc3.bias,args)
        #print("act2 successed")
        
        act3 = flp2fixTensor(act2, args.full_bits, args.frac_bits)
        #print("act3 successed")
    return cmodel, act0, act1, act2, act3 

In [38]:
def testQuant(model, test_loader, args):
    
    start = time.time()
    
    qmodel = copy.deepcopy(model).to(args.device)
    qmodel = model2fix(qmodel, args)
    qmodel.eval()
    
    with torch.no_grad():
        loss_func = genLossFunc(args)
        loss, correct = 0, 0
        for batch_index, (image, label) in enumerate(tq(test_loader,desc='Test',leave=False)):
            start = time.time()
            image, label = image.to(args.device), label.to(args.device)
            qmodel, act0, act1, act2, act3  = quantFixForward(qmodel, image, args)
            y = act3
            loss += loss_func(y, label).item()#*image.size(0)
            correct += (y.argmax(1) == label).type(torch.int).sum().item()
            end = time.time()
            sec = end - start
            result_list = str(datetime.timedelta(seconds=sec)).split(".")
            print(f'image {batch_index} time  : {result_list[0]}')
    correct_rate = 100 * correct / len(test_loader.dataset)
    print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%) Loss: {loss/len(test_loader.dataset):.2f}')
    
    end = time.time()
    sec = (end-start)
    result_list = str(datetime.timedelta(seconds=sec)).split(".")
    print(f'Total time is : {result_list[0]}')
    return qmodel, act0, act1, act2, act3

In [39]:
qmodel, act0, act1, act2, act3 = testQuant(model, test_loader, args)

Test:   0%|          | 0/157 [00:00<?, ?it/s]

image 0 time  : 0:01:52
image 1 time  : 0:01:53
image 2 time  : 0:01:51
image 3 time  : 0:01:52
image 4 time  : 0:01:52
image 5 time  : 0:01:52
image 6 time  : 0:01:52
image 7 time  : 0:01:52
image 8 time  : 0:01:52
image 9 time  : 0:01:51
image 10 time  : 0:01:53
image 11 time  : 0:01:52
image 12 time  : 0:01:52
image 13 time  : 0:01:51
image 14 time  : 0:01:52
image 15 time  : 0:01:52
image 16 time  : 0:01:52
image 17 time  : 0:01:52
image 18 time  : 0:01:53
image 19 time  : 0:01:52
image 20 time  : 0:01:52
image 21 time  : 0:01:52
image 22 time  : 0:01:53
image 23 time  : 0:01:53
image 24 time  : 0:01:53
image 25 time  : 0:01:52
image 26 time  : 0:01:52
image 27 time  : 0:01:53
image 28 time  : 0:01:52
image 29 time  : 0:01:52
image 30 time  : 0:01:52
image 31 time  : 0:01:53
image 32 time  : 0:01:52
image 33 time  : 0:01:53
image 34 time  : 0:01:52
image 35 time  : 0:01:53
1b10111011111111
1b11101011000011
10b1001001110110
1b10001101010010
1000b10101111110
100b111010010010
100b1100